In [1]:
%cd ../

/home/qwj/code/HippoRAG


In [10]:
import pandas as pd
import subprocess
import os
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [12]:
def calculate_recall_mean(output_path):
    result = pd.read_json(output_path)
    def recall_dict_to_columns(recall_dict):
        return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})
    recall_df = result['recall'].apply(recall_dict_to_columns)
    return recall_df.mean()

To get familiar with the RETRIEVAL_MODULE

## Create Graph

In [3]:
import os
import shutil

def is_new_files(file_name):
    return not (file_name.startswith("openie_") or file_name.startswith('.') or file_name.endswith("named_entity_output.tsv") or file_name.endswith("entity_and_relations.tsv"))

def move_files(subfolder_name):
    # 定义新子文件夹的名称
    new_subfolder = os.path.join('output', subfolder_name)
    
    # 如果新子文件夹不存在，创建它
    if not os.path.exists(new_subfolder):
        os.makedirs(new_subfolder)

    # 获取文件夹下的所有文件（不包含子文件夹）
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
        
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                # 移动文件到新子文件夹
                print(f"Moved: {file_name}")
                shutil.move(file_path, os.path.join(new_subfolder, file_name))

def show_new_files():
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
    
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                print(file_name)

def remove_new_files():
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)

        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                print(f"Removed: {file_name}")
                os.remove(file_path)


In [4]:
remove_new_files()
show_new_files()

Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_similarity_edges_mean_0.7_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_mean_0.7_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_

In [5]:
!CUDA_VISIBLE_DEVICES="4" python src/create_graph.py \
    --dataset musique \
    --model_name facebook/contriever \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner

100%|███████████████████████████████████| 11656/11656 [00:01<00:00, 9811.51it/s]
Correct Wiki Format: 0 out of 11656
100%|████████████████████████████████████| 1006/1006 [00:00<00:00, 65784.28it/s]


In [6]:
show_new_files()

query_to_kb.tsv
rel_kb_to_kb.tsv
kb_to_kb.tsv


### Index by contriever

In [7]:
!CUDA_VISIBLE_DEVICES="4" python src/RetrievalModule.py --retriever_name facebook/contriever --string_filename output/query_to_kb.tsv

No Pre-Computed Vectors. Confirming PLM Model.
Loading PLM Vectors.
100%|████████████████████████████████| 92963/92963 [00:00<00:00, 2891901.53it/s]
Populating Vector Dict
100%|██████████████████████████████████| 92963/92963 [00:03<00:00, 25443.41it/s]
Vectors Loaded.
Chunking
Building Index
Running Index Part 0

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 295.02it/s]
Running Index Part 1

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 372.68it/s]
Running Index Part 2

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 366.73it/s]
Running Index Part 3

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 375.35it/s]
Combining Index Chunks
1234it [00:00, 5869.24it/s]
1234it [00:00, 3763.63it/s]


In [13]:
!CUDA_VISIBLE_DEVICES="4,5" python src/RetrievalModule.py --retriever_name facebook/contriever --string_filename output/kb_to_kb.tsv

No Pre-Computed Vectors. Confirming PLM Model.
Loading PLM Vectors.
100%|██████████████████████████████| 183458/183458 [00:00<00:00, 2907247.44it/s]
Populating Vector Dict
100%|████████████████████████████████| 183458/183458 [00:06<00:00, 26259.03it/s]
Vectors Loaded.
Chunking
Building Index
Running Index Part 0

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 46.12it/s]
Running Index Part 1

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 45.99it/s]
Running Index Part 2

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 47.20it/s]
Running Index Part 3

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 52.17it/s]
Combining Index Chunks
91729it [00:15, 5746.26it/s]
91729it [00:23, 3878.19it/s]


In [14]:
!CUDA_VISIBLE_DEVICES="4,5" python src/RetrievalModule.py --retriever_name facebook/contriever --string_filename output/rel_kb_to_kb.tsv

No Pre-Computed Vectors. Confirming PLM Model.
Loading PLM Vectors.
100%|████████████████████████████████| 43428/43428 [00:00<00:00, 2863592.17it/s]
Encoding 21377 Missing Strings
100%|███████████████████████████████████| 21377/21377 [00:03<00:00, 5846.28it/s]
3it [00:00,  3.36it/s]
Populating Vector Dict
100%|██████████████████████████████████| 43428/43428 [00:01<00:00, 25736.81it/s]
Vectors Loaded.
Chunking
Building Index
Running Index Part 0

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 188.62it/s]
Running Index Part 1

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 191.70it/s]
Running Index Part 2

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 192.34it/s]
Running Index Part 3

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 200.26it/s]
Combining Index Chunks
21714it [00:03, 6311.68it/s]
21714it [00:05, 4311.91it/s]


In [8]:
remove_new_files()
!CUDA_VISIBLE_DEVICES="4" python src/create_graph.py \
    --dataset musique \
    --model_name facebook/contriever \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner \
    --create_graph \
    --cosine_sim_edges

Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: kb_to_kb.tsv
100%|███████████████████████████████████| 11656/11656 [00:01<00:00, 9431.26it/s]
Correct Wiki Format: 0 out of 11656
100%|████████████████████████████████████| 1006/1006 [00:00<00:00, 55028.43it/s]
Creating Graph
100%|██████████████████████████████████| 11656/11656 [00:00<00:00, 11813.48it/s]
Loading Vectors
Augmenting Graph from Similarity
100%|██████████████████████████████████| 91729/91729 [00:03<00:00, 28135.31it/s]
Saving Graph
                                                         1
0                                                         
Total Phrases                                       327435
Unique Phrases                                       91729
Number of Individual Triples                        109145
Number of Incorrectly Formatted Triples (ChatGP...    1136
Number of Triples w/o NER Entities (ChatGPT Error)    6762
Number of Unique Individual Triples                 107448
Number of Entities

In [16]:
show_new_files()

musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_similarity_edges_mean_0.8_thresh_ents_only_lower_preprocess_ner_facebook_contriever.v3.subset.p
musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_facts_and_sim_graph_mean_0.8_thresh_ents_only_lower_preprocess_ner_facebook_contriever.v3.subset.p
query_to_kb.tsv
rel_kb_to_kb.tsv
musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
kb_to_kb.tsv
musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_facebook_contriever.v3.subset.p
musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
musiq

In [ ]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

# Run the script with the specified arguments
!python3 src/ircot_hipporag.py --dataset musique --retriever facebook/contriever --max_steps 1 --doc_ensemble t --top_k 10 --sim_threshold 0.8 --damping 0.5


In [19]:
output_path = "output/ircot/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json"

result = pd.read_json(output_path)

def recall_dict_to_columns(recall_dict):
    return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})

# 对每一行的 recall 列应用上述函数，并将结果拼接到新的 DataFrame
recall_df = result['recall'].apply(recall_dict_to_columns)

# 显示新创建的 DataFrame
recall_df.mean()

recall@1      0.290575
recall@2      0.423080
recall@5      0.544833
recall@10     0.626001
recall@15     0.626001
recall@20     0.626001
recall@30     0.626001
recall@40     0.626001
recall@50     0.626001
recall@80     0.626001
recall@100    0.626001
dtype: float64

In [10]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

# Run the script with the specified arguments
!python3 src/ircot_hipporag.py --dataset musique --retriever facebook/contriever --max_steps 1 --doc_ensemble f --top_k 10 --sim_threshold 0.8 --damping 0.5


Building Graph: 100%|███████████████| 349755/349755 [00:00<00:00, 559509.82it/s]
2024-07-10 08:14:53,234 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 349644
2024-07-10 08:14:53,259 - hipporag - INFO - Loading node vectors from: data/lm_vectors/facebook_contriever_mean/encoded_strings.txt
2024-07-10 08:14:53,893 - hipporag - INFO - 1 phrases did not have vectors.
[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json
HF model facebook/contriever encoding: 100%|█████| 2/2 [00:00<00:00, 147.20it/s]

HF model facebook/contriever encoding: 100%|█████| 2/2 [00:00<00:00, 108.44it/s]

HF model facebook/contriever encoding: 100%|█████|

In [11]:
output_path = "output/ircot/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json"

result = pd.read_json(output_path)

def recall_dict_to_columns(recall_dict):
    return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})

# 对每一行的 recall 列应用上述函数，并将结果拼接到新的 DataFrame
recall_df = result['recall'].apply(recall_dict_to_columns)

# 显示新创建的 DataFrame
recall_df.mean()

recall@1      0.283992
recall@2      0.409579
recall@5      0.520830
recall@10     0.589332
recall@15     0.589332
recall@20     0.589332
recall@30     0.589332
recall@40     0.589332
recall@50     0.589332
recall@80     0.589332
recall@100    0.589332
dtype: float64

## GRIT

In [5]:
remove_new_files()
!CUDA_VISIBLE_DEVICES="4" python src/create_graph.py \
    --dataset musique \
    --model_name "grit-7b" \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner

Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_similarity_edges_mean_0.8_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_s

In [6]:
show_new_files()

query_to_kb.tsv
rel_kb_to_kb.tsv
kb_to_kb.tsv


In [7]:
pd.read_csv("output/kb_to_kb.tsv", sep="\t").head()

,Unnamed: 0,strings,type
0,0,NaN,query
1,1,0,query
2,2,0 0,query
3,3,0 2,query
4,4,0 1 972,query


In [8]:
len(pd.read_csv("output/kb_to_kb.tsv", sep="\t"))

183458

In [ ]:
!CUDA_VISIBLE_DEVICES="2,3" python src/grit_retrieval_module.py --retriever_name "grit-7b" --string_filename output/query_to_kb.tsv --query_type "entity"

In [ ]:
!CUDA_VISIBLE_DEVICES="4,5" python src/grit_retrieval_module.py --retriever_name "grit-7b" --string_filename output/kb_to_kb.tsv --query_type "entity"

In [ ]:
!CUDA_VISIBLE_DEVICES="6,7" python src/grit_retrieval_module.py --retriever_name "grit-7b" --string_filename output/rel_kb_to_kb.tsv --query_type "entity"

Threshold 0.8

In [10]:
remove_new_files()
!CUDA_VISIBLE_DEVICES="4" python src/create_graph.py \
    --dataset musique \
    --model_name "grit-7b" \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner \
    --create_graph \
    --cosine_sim_edges
show_new_files()

Removed: musique_facts_and_sim_graph_mean_0.6_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_similarity_edges_mean_0.6_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_

In [11]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

# Run the script with the specified arguments
!python3 src/ircot_hipporag.py --dataset musique --retriever "grit-7b" --max_steps 1 --doc_ensemble t --top_k 10 --sim_threshold 0.8 --damping 0.5 --recognition_threshold 0.7

Loading checkpoint shards: 100%|██████████████████| 3/3 [00:03<00:00,  1.05s/it]
Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn
Building Graph: 100%|███████████████| 206897/206897 [00:00<00:00, 519413.87it/s]
2024-07-10 04:10:24,723 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 206786
2024-07-10 04:10:24,741 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-10 04:10:27,053 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-10 04:10:27,176 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)
[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.7_step_1_top_10_sim_thresh_0.8_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_

In [12]:
output_path = "output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.7_step_1_top_10_sim_thresh_0.8_damping_0.5.json"

result = pd.read_json(output_path)

def recall_dict_to_columns(recall_dict):
    return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})

# 对每一行的 recall 列应用上述函数，并将结果拼接到新的 DataFrame
recall_df = result['recall'].apply(recall_dict_to_columns)

# 显示新创建的 DataFrame
recall_df.mean()

recall@1      0.241910
recall@2      0.362830
recall@5      0.520083
recall@10     0.614168
recall@15     0.614168
recall@20     0.614168
recall@30     0.614168
recall@40     0.614168
recall@50     0.614168
recall@80     0.614168
recall@100    0.614168
dtype: float64

Threshold 0.75

In [8]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

remove_new_files()
!python3 src/create_graph.py \
    --dataset musique \
    --model_name "grit-7b" \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.7 \
    --extraction_type ner \
    --create_graph \
    --cosine_sim_edges
show_new_files()

# Run the script with the specified arguments
!python3 src/ircot_hipporag.py --dataset musique --retriever "grit-7b" --max_steps 1 --doc_ensemble t --top_k 10 --sim_threshold 0.7 --damping 0.5

sim_threshold = 0.7
output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"

result = pd.read_json(output_path)

def recall_dict_to_columns(recall_dict):
    return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})

# 对每一行的 recall 列应用上述函数，并将结果拼接到新的 DataFrame
recall_df = result['recall'].apply(recall_dict_to_columns)

# 显示新创建的 DataFrame
recall_df.mean()

Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_mean_0.75_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_a

recall@1      0.249660
recall@2      0.385330
recall@5      0.566251
recall@10     0.677087
recall@15     0.677087
recall@20     0.677087
recall@30     0.677087
recall@40     0.677087
recall@50     0.677087
recall@80     0.677087
recall@100    0.677087
dtype: float64

In [9]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

remove_new_files()
!python3 src/create_graph.py \
    --dataset musique \
    --model_name "grit-7b" \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.6 \
    --extraction_type ner \
    --create_graph \
    --cosine_sim_edges
show_new_files()

# Run the script with the specified arguments
!python3 src/ircot_hipporag.py --dataset musique --retriever "grit-7b" --max_steps 1 --doc_ensemble t --top_k 10 --sim_threshold 0.6 --damping 0.5

sim_threshold = 0.6
output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"

result = pd.read_json(output_path)

def recall_dict_to_columns(recall_dict):
    return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})

# 对每一行的 recall 列应用上述函数，并将结果拼接到新的 DataFrame
recall_df = result['recall'].apply(recall_dict_to_columns)

# 显示新创建的 DataFrame
recall_df.mean()

Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_similarity_edges_mean_0.7_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_mean_0.7_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_

recall@1      0.253826
recall@2      0.391997
recall@5      0.567335
recall@10     0.678337
recall@15     0.678337
recall@20     0.678337
recall@30     0.678337
recall@40     0.678337
recall@50     0.678337
recall@80     0.678337
recall@100    0.678337
dtype: float64

## HypeScan

In [10]:

sim_thresholds = [0.8, 0.7, 0.6]
recognition_thresholds = [0.8, 0.7, 0.6]

def calculate_recall_mean(output_path):
    result = pd.read_json(output_path)
    def recall_dict_to_columns(recall_dict):
        return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})
    recall_df = result['recall'].apply(recall_dict_to_columns)
    return recall_df.mean()

# Store results
results = []

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

# Run experiments
for sim_threshold in sim_thresholds:
    print("sim_threshold", sim_threshold)
    remove_new_files()
    create_graph_command = [
        'python3', 'src/create_graph.py',
        '--dataset', 'musique',
        '--model_name', 'grit-7b',
        '--extraction_model', 'gpt-3.5-turbo-1106',
        '--threshold', str(sim_threshold),
        '--extraction_type', 'ner',
        '--create_graph', '--cosine_sim_edges'
    ]
    subprocess.run(create_graph_command, check=True)
    for recognition_threshold in recognition_thresholds:
        print("recognition_threshold", recognition_threshold)
        ircot_hipporag_command = [
            'python3', 'src/ircot_hipporag_v2.py',
            '--dataset', 'musique',
            '--retriever', 'grit-7b',
            '--max_steps', '1',
            '--doc_ensemble', 't',
            '--top_k', '10',
            '--sim_threshold', str(sim_threshold),
            '--damping', '0.5',
            '--recognition_threshold', str(recognition_threshold)
        ]
        subprocess.run(ircot_hipporag_command, check=True)
        
        # Define output path
        output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_{recognition_threshold}_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"
        
        # Calculate recall mean
        recall_mean = calculate_recall_mean(output_path)
        
        # Store the result
        results.append({
            'sim_threshold': sim_threshold,
            'recognition_threshold': recognition_threshold,
            'recall_mean': recall_mean
        })

sim_threshold 0.8


100%|██████████| 1006/1006 [00:00<00:00, 70558.52it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:00<00:00, 12158.79it/s]


Loading Vectors
Augmenting Graph from Similarity


...

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 271371/271371 [00:00<00:00, 599137.40it/s]
2024-07-10 06:59:00,433 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 271260
2024-07-10 06:59:00,454 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-10 06:59:02,709 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-10 06:59:02,841 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)


[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.6_step_1_top_10_sim_thresh_0.6_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.6_step_1_top_10_sim_thresh_0.6_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [02:55<00:00,  5.68it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.6_step_1_top_10_sim_thresh_0.6_damping_0.5.json


In [11]:
results

[{'sim_threshold': 0.8,
  'recognition_threshold': 0.8,
  'recall_mean': recall@1      0.245826
  recall@2      0.377747
  recall@5      0.554167
  recall@10     0.665003
  recall@15     0.665003
  recall@20     0.665003
  recall@30     0.665003
  recall@40     0.665003
  recall@50     0.665003
  recall@80     0.665003
  recall@100    0.665003
  dtype: float64},
 {'sim_threshold': 0.8,
  'recognition_threshold': 0.7,
  'recall_mean': recall@1      0.241910
  recall@2      0.362830
  recall@5      0.520083
  recall@10     0.614168
  recall@15     0.614168
  recall@20     0.614168
  recall@30     0.614168
  recall@40     0.614168
  recall@50     0.614168
  recall@80     0.614168
  recall@100    0.614168
  dtype: float64},
 {'sim_threshold': 0.8,
  'recognition_threshold': 0.6,
  'recall_mean': recall@1      0.232910
  recall@2      0.347412
  recall@5      0.466581
  recall@10     0.534082
  recall@15     0.534082
  recall@20     0.534082
  recall@30     0.534082
  recall@40     0.534082

In [12]:
import subprocess


def calculate_recall_mean(output_path):
    result = pd.read_json(output_path)
    def recall_dict_to_columns(recall_dict):
        return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})
    recall_df = result['recall'].apply(recall_dict_to_columns)
    return recall_df.mean()

# Store results
results2 = []

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'

experiments2 = {
    0.8: [0.925, 0.95],
    0.7: [0.9, 0.925, 0.95],
    0.6: [0.9, 0.925, 0.95],
}

# Run experiments
for sim_threshold, recognition_threshold_list in experiments2.items():
    print("sim_threshold", sim_threshold)
    remove_new_files()
    create_graph_command = [
        'python3', 'src/create_graph.py',
        '--dataset', 'musique',
        '--model_name', 'grit-7b',
        '--extraction_model', 'gpt-3.5-turbo-1106',
        '--threshold', str(sim_threshold),
        '--extraction_type', 'ner',
        '--create_graph', '--cosine_sim_edges'
    ]
    subprocess.run(create_graph_command, check=True)
    for recognition_threshold in recognition_threshold_list:
        print("recognition_threshold", recognition_threshold)
        ircot_hipporag_command = [
            'python3', 'src/ircot_hipporag.py',
            '--dataset', 'musique',
            '--retriever', 'grit-7b',
            '--max_steps', '1',
            '--doc_ensemble', 't',
            '--top_k', '10',
            '--sim_threshold', str(sim_threshold),
            '--damping', '0.5',
            '--recognition_threshold', str(recognition_threshold)
        ]
        subprocess.run(ircot_hipporag_command, check=True)
        
        # Define output path
        output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_{recognition_threshold}_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"
        
        # Calculate recall mean
        recall_mean = calculate_recall_mean(output_path)
        
        # Store the result
        results2.append({
            'sim_threshold': sim_threshold,
            'recognition_threshold': recognition_threshold,
            'recall_mean': recall_mean
        })

sim_threshold 0.8
Removed: musique_facts_and_sim_graph_mean_0.6_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_similarity_edges_mean_0.6_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p

100%|██████████| 1006/1006 [00:00<00:00, 71089.90it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:00<00:00, 12033.01it/s]


Loading Vectors
Augmenting Graph from Similarity


...

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 271371/271371 [00:00<00:00, 614538.12it/s]
2024-07-10 07:43:01,514 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 271260
2024-07-10 07:43:01,533 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-10 07:43:03,853 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-10 07:43:03,991 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)


[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.95_step_1_top_10_sim_thresh_0.6_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.95_step_1_top_10_sim_thresh_0.6_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [03:10<00:00,  5.24it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.95_step_1_top_10_sim_thresh_0.6_damping_0.5.json


In [13]:
results2

[{'sim_threshold': 0.8,
  'recognition_threshold': 0.925,
  'recall_mean': recall@1      0.247577
  recall@2      0.380496
  recall@5      0.563501
  recall@10     0.674087
  recall@15     0.674087
  recall@20     0.674087
  recall@30     0.674087
  recall@40     0.674087
  recall@50     0.674087
  recall@80     0.674087
  recall@100    0.674087
  dtype: float64},
 {'sim_threshold': 0.8,
  'recognition_threshold': 0.95,
  'recall_mean': recall@1      0.247577
  recall@2      0.380496
  recall@5      0.563501
  recall@10     0.674087
  recall@15     0.674087
  recall@20     0.674087
  recall@30     0.674087
  recall@40     0.674087
  recall@50     0.674087
  recall@80     0.674087
  recall@100    0.674087
  dtype: float64},
 {'sim_threshold': 0.7,
  'recognition_threshold': 0.9,
  'recall_mean': recall@1      0.249660
  recall@2      0.385330
  recall@5      0.566251
  recall@10     0.677087
  recall@15     0.677087
  recall@20     0.677087
  recall@30     0.677087
  recall@40     0.677

In [10]:


# Store results
results3 = []

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'


# Run experiments
for sim_threshold in [0.8, 0.7, 0.6]:
    print("sim_threshold", sim_threshold)
    # remove_new_files()
    create_graph_command = [
        'python3', 'src/create_graph.py',
        '--dataset', 'musique',
        '--model_name', 'grit-7b',
        '--extraction_model', 'gpt-3.5-turbo-1106',
        '--threshold', str(sim_threshold),
        '--extraction_type', 'ner',
        '--create_graph', '--cosine_sim_edges'
    ]
    subprocess.run(create_graph_command, check=True)
    ircot_hipporag_command = [
        'python3', 'src/ircot_hipporag.py',
        '--dataset', 'musique',
        '--retriever', 'grit-7b',
        '--max_steps', '1',
        '--doc_ensemble', 'f',
        '--top_k', '10',
        '--sim_threshold', str(sim_threshold),
        '--damping', '0.5',
    ]
    subprocess.run(ircot_hipporag_command, check=True)
    
    # Define output path

    output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"
    
    # Calculate recall mean
    recall_mean = calculate_recall_mean(output_path)
    
    # Store the result
    results3.append({
        'sim_threshold': sim_threshold,
        # 'recognition_threshold': recognition_threshold,
        'recall_mean': recall_mean
    })

sim_threshold 0.8


100%|██████████| 1006/1006 [00:00<00:00, 70597.47it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:01<00:00, 11645.64it/s]


Loading Vectors
Augmenting Graph from Similarity


...

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 271371/271371 [00:00<00:00, 635825.65it/s]
2024-07-11 14:02:34,776 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 271260
2024-07-11 14:02:34,794 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-11 14:02:36,905 - hipporag - INFO - 1 phrases did not have vectors.
IRCoT retrieval:   0%|          | 0/1000 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.6_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.6_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [02:15<00:00,  7.39it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.6_damping_0.5.json


In [13]:
results3 = []
for sim_threshold in [0.575, 0.55, 0.525, 0.5, 0.475]:
    print("sim_threshold", sim_threshold)
    # remove_new_files()
    create_graph_command = [
        'python3', 'src/create_graph.py',
        '--dataset', 'musique',
        '--model_name', 'grit-7b',
        '--extraction_model', 'gpt-3.5-turbo-1106',
        '--threshold', str(sim_threshold),
        '--extraction_type', 'ner',
        '--create_graph', '--cosine_sim_edges'
    ]
    subprocess.run(create_graph_command, check=True)
    ircot_hipporag_command = [
        'python3', 'src/ircot_hipporag.py',
        '--dataset', 'musique',
        '--retriever', 'grit-7b',
        '--max_steps', '1',
        '--doc_ensemble', 'f',
        '--top_k', '10',
        '--sim_threshold', str(sim_threshold),
        '--damping', '0.5',
    ]
    subprocess.run(ircot_hipporag_command, check=True)
    
    # Define output path

    output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"
    
    # Calculate recall mean
    recall_mean = calculate_recall_mean(output_path)
    
    # Store the result
    results3.append({
        'sim_threshold': sim_threshold,
        # 'recognition_threshold': recognition_threshold,
        'recall_mean': recall_mean
    })

sim_threshold 0.575


100%|██████████| 1006/1006 [00:00<00:00, 65919.95it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:00<00:00, 12072.26it/s]


Loading Vectors
Augmenting Graph from Similarity


...

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 1105803/1105803 [00:01<00:00, 588168.81it/s]
2024-07-12 04:27:35,330 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 1105692
2024-07-12 04:27:35,417 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-12 04:27:37,546 - hipporag - INFO - 1 phrases did not have vectors.


Loaded 1000 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.475_damping_0.5.json
R@1: 0.2687 R@2: 0.3867 R@5: 0.5072 R@10: 0.5823 R@15: 0.5823 R@20: 0.5823 R@30: 0.5823 R@40: 0.5823 R@50: 0.5823 R@80: 0.5823 R@100: 0.5823 


IRCoT retrieval: 100%|██████████| 1000/1000 [00:00<00:00, 1178175.28it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.475_damping_0.5.json


In [14]:
results3

[{'sim_threshold': 0.575,
  'recall_mean': recall@1      0.258826
  recall@2      0.379163
  recall@5      0.505331
  recall@10     0.568749
  recall@15     0.568749
  recall@20     0.568749
  recall@30     0.568749
  recall@40     0.568749
  recall@50     0.568749
  recall@80     0.568749
  recall@100    0.568749
  dtype: float64},
 {'sim_threshold': 0.55,
  'recall_mean': recall@1      0.262076
  recall@2      0.384579
  recall@5      0.508998
  recall@10     0.579166
  recall@15     0.579166
  recall@20     0.579166
  recall@30     0.579166
  recall@40     0.579166
  recall@50     0.579166
  recall@80     0.579166
  recall@100    0.579166
  dtype: float64},
 {'sim_threshold': 0.525,
  'recall_mean': recall@1      0.263909
  recall@2      0.383663
  recall@5      0.509665
  recall@10     0.585333
  recall@15     0.585333
  recall@20     0.585333
  recall@30     0.585333
  recall@40     0.585333
  recall@50     0.585333
  recall@80     0.585333
  recall@100    0.585333
  dtype: float6

In [13]:
results4=[]
for sim_threshold in [0.8, 0.7, 0.6]:
    print("sim_threshold", sim_threshold)
    # remove_new_files()
    # create_graph_command = [
    #     'python3', 'src/create_graph.py',
    #     '--dataset', 'musique',
    #     '--model_name', 'grit-7b',
    #     '--extraction_model', 'gpt-3.5-turbo-1106',
    #     '--threshold', str(sim_threshold),
    #     '--extraction_type', 'ner',
    #     '--create_graph', '--cosine_sim_edges'
    # ]
    # subprocess.run(create_graph_command, check=True)
    ircot_hipporag_command = [
        'python3', 'src/ircot_hipporag.py',
        '--dataset', 'musique',
        '--retriever', 'grit-7b',
        '--max_steps', '1',
        '--doc_ensemble', 't',
        '--top_k', '10',
        '--sim_threshold', str(sim_threshold),
        '--damping', '0.5',
        '--recognition_threshold', '0.9'
    ]
    subprocess.run(ircot_hipporag_command, check=True)
    
    # Define output path
    output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"

    
    # Calculate recall mean
    recall_mean = calculate_recall_mean(output_path)
    
    # Store the result
    results4.append({
        'filename': output_path,
        'sim_threshold': sim_threshold,
        'recognition_threshold': 0.9,
        'recall_mean': recall_mean
    })

sim_threshold 0.8


Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 206897/206897 [00:00<00:00, 579132.10it/s]
2024-07-11 14:17:48,055 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 206786
2024-07-11 14:17:48,072 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-11 14:17:50,231 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-11 14:17:50,359 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)


Loaded 1000 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json
R@1: 0.2708 R@2: 0.4002 R@5: 0.5841 R@10: 0.6842 R@15: 0.6842 R@20: 0.6842 R@30: 0.6842 R@40: 0.6842 R@50: 0.6842 R@80: 0.6842 R@100: 0.6842 


...

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 271371/271371 [00:00<00:00, 620603.95it/s]
2024-07-11 14:20:59,321 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 271260
2024-07-11 14:20:59,339 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-11 14:21:01,510 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-11 14:21:01,630 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)


[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.6_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.6_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [02:52<00:00,  5.80it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.6_damping_0.5.json


In [16]:
for sim_threshold in [0.5, 0.4]:
    print("sim_threshold", sim_threshold)
    # remove_new_files()
    create_graph_command = [
        'python3', 'src/create_graph.py',
        '--dataset', 'musique',
        '--model_name', 'grit-7b',
        '--extraction_model', 'gpt-3.5-turbo-1106',
        '--threshold', str(sim_threshold),
        '--extraction_type', 'ner',
        '--create_graph', '--cosine_sim_edges'
    ]
    subprocess.run(create_graph_command, check=True)
    ircot_hipporag_command = [
        'python3', 'src/ircot_hipporag.py',
        '--dataset', 'musique',
        '--retriever', 'grit-7b',
        '--max_steps', '1',
        '--doc_ensemble', 't',
        '--top_k', '10',
        '--sim_threshold', str(sim_threshold),
        '--damping', '0.5',
        '--recognition_threshold', '0.9'
    ]
    subprocess.run(ircot_hipporag_command, check=True)
    
    # Define output path
    output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"

    
    # Calculate recall mean
    recall_mean = calculate_recall_mean(output_path)
    
    # Store the result
    results4.append({
        'filename': output_path,
        'sim_threshold': sim_threshold,
        'recognition_threshold': 0.9,
        'recall_mean': recall_mean
    })

sim_threshold 0.5


100%|██████████| 1006/1006 [00:00<00:00, 71588.03it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:00<00:00, 11798.30it/s]


Loading Vectors
Augmenting Graph from Similarity


...

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn


Building Graph: 100%|██████████| 3599197/3599197 [00:07<00:00, 500500.38it/s]
2024-07-11 14:41:31,538 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 3599086
2024-07-11 14:41:31,839 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-11 14:41:34,072 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-11 14:41:34,192 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)


[Errno 2] No such file or directory: 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.4_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.4_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [12:12<00:00,  1.37it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.4_damping_0.5.json


In [11]:
results4 = []
for sim_threshold in [0.525, 0.55, 0.575]:
    print("sim_threshold", sim_threshold)
    # remove_new_files()
    create_graph_command = [
        'python3', 'src/create_graph.py',
        '--dataset', 'musique',
        '--model_name', 'grit-7b',
        '--extraction_model', 'gpt-3.5-turbo-1106',
        '--threshold', str(sim_threshold),
        '--extraction_type', 'ner',
        '--create_graph', '--cosine_sim_edges'
    ]
    subprocess.run(create_graph_command, check=True)
    ircot_hipporag_command = [
        'python3', 'src/ircot_hipporag.py',
        '--dataset', 'musique',
        '--retriever', 'grit-7b',
        '--max_steps', '1',
        '--doc_ensemble', 't',
        '--top_k', '10',
        '--sim_threshold', str(sim_threshold),
        '--damping', '0.5',
        '--recognition_threshold', '0.9'
    ]
    subprocess.run(ircot_hipporag_command, check=True)
    
    # Define output path
    output_path = f"output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_{sim_threshold}_damping_0.5.json"

    
    # Calculate recall mean
    recall_mean = calculate_recall_mean(output_path)
    
    # Store the result
    results4.append({
        'filename': output_path,
        'sim_threshold': sim_threshold,
        'recognition_threshold': 0.9,
        'recall_mean': recall_mean
    })

sim_threshold 0.525


  0%|          | 0/1006 [00:00<?, ?it/s]

Correct Wiki Format: 0 out of 11656


100%|██████████| 1006/1006 [00:00<00:00, 65737.14it/s]


Creating Graph


...

Building Graph: 100%|██████████| 524394/524394 [00:00<00:00, 599080.04it/s]
2024-07-12 04:16:18,259 - hipporag - INFO - Graph built: num vertices: 91729, num_edges: 524283
2024-07-12 04:16:18,312 - hipporag - INFO - Loading node vectors from: data/lm_vectors/grit-7b_mean/encoded_strings.txt
2024-07-12 04:16:20,516 - hipporag - INFO - 1 phrases did not have vectors.
2024-07-12 04:16:20,640 - hipporag - INFO - Loaded doc embeddings from data/lm_vectors/grit-7b_mean/musique_doc_embeddings.p, shape: (11656, 4096)


Loaded 1000 results from output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.525_damping_0.5.json
R@1: 0.2759 R@2: 0.4233 R@5: 0.5882 R@10: 0.6935 R@15: 0.6935 R@20: 0.6935 R@30: 0.6935 R@40: 0.6935 R@50: 0.6935 R@80: 0.6935 R@100: 0.6935 


IRCoT retrieval: 100%|██████████| 1000/1000 [00:00<00:00, 1164760.90it/s]


Saved 1000 results to output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.525_damping_0.5.json


NameError: name 'calculate_recall_mean' is not defined

In [19]:
results4

[{'filename': 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json',
  'sim_threshold': 0.8,
  'recognition_threshold': 0.9,
  'recall_mean': recall@1      0.270825
  recall@2      0.400246
  recall@5      0.584085
  recall@10     0.684171
  recall@15     0.684171
  recall@20     0.684171
  recall@30     0.684171
  recall@40     0.684171
  recall@50     0.684171
  recall@80     0.684171
  recall@100    0.684171
  dtype: float64},
 {'filename': 'output/ircot/ircot_results_musique_hipporag_grit-7b_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.7_damping_0.5.json',
  'sim_threshold': 0.7,
  'recognition_threshold': 0.9,
  'recall_mean': recall@1      0.272159
  recall@2      0.407246
  recall@5      0.586419
  recall@10     0.684921
  recall@15     0.684921
  recall@20     0.684921
  recall@30     0.684921
  recall@40     0.684921
  recall@50     0.684921
  recall@80     0.68492